<a href="https://colab.research.google.com/github/Jose-Gabriel-Rodriguez/MachineLearning/blob/main/Unidad3/Practica_2_Regresion_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://github.com/JoseGabriel-ITD/Probabilidad-y-Estadistica/blob/main/Cintilla2004.png?raw=true)

# **Machine Learning y Deep Learning**

## Unidad 3

### Practica 2: **Regresión Logística**

### Facilitador: *Dr. José Gabriel Rodríguez Rivas*

### Alumno:  _____________


## **Regresión logística**  

- La regresión logística toma un conjunto de variables de entrada, las características (variables) , y estima un valor objetivo.
- La regresión logística es similar a la regresión lineal, la diferencia consiste en que **el modelo de regresión logística además calcula una suma ponderada de la entrada características xi y el término de interceptación b**, pero ejecuta este resultado a través de una función no lineal f,  para producir la salida **yhat** (predicción).
- El efecto de aplicar la función logística es comprimir la salida de la función lineal para que se limite a un rango entre 0 y 1.
- La salida de la fórmula de regresión logística, puede interpretarse como **la probabilidad de que la instancia de datos de entrada pertenezca a la clase positiva, dadas sus características de entrada**.




## **Contexto de la práctica**    

- En la primera parte, se enfocará en **comprender los objetivos y requisitos del dataset a utilizar desde una perspectiva comercial**, comenzando con la comprensión de cómo funciona el **club de prestamos Lending Club**.

### **Club de préstamos**

LendingClub es una compañía estadounidense de **préstamos entre pares**, con sede en San Francisco, California. Es la plataforma de préstamos peer-to-peer más grande del mundo.

**LendingClub**
- permite a los **prestatarios solicitar préstamos personales no garantizados** entre  1,000 y 40,000.
- Los **inversionistas pueden buscar y explorar las listas de préstamos** en el sitio web de LendingClub **y seleccionar los préstamos en los que desean invertir según la información proporcionada sobre el prestatario, el monto del préstamo, el grado del préstamo y el propósito del préstamo**, con una inversión mínima de $25.
- Los inversionistas ganan dinero con los intereses.
- LendingClub gana dinero cobrando a los prestatarios una tarifa de originación y a los inversionistas una tarifa de servicio.

Para obtener más información sobre la empresa, consulte el artículo de wikipedia sobre [LendingClub](https://en.wikipedia.org/wiki/LendingClub).



In [59]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Importamos librerias necesarias para regresion logistica y evaluacion de modelos
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

In [60]:
prestamos_df = pd.read_csv('prestamos_ok.csv')
prestamos_df.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,issue_year,repaid,loan_term_year,purpose_code,home_ownership_code,grade_code,addr_state_code
0,0,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,Cash,N,NaN,2011,1,3,11,4,2,2
1,1,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,Cash,N,NaN,2011,1,3,9,4,2,0
2,2,3000,3000,3000.0,36 months,18.64,109.43,E,E1,MKC Accounting,...,Cash,N,NaN,2011,1,3,0,4,4,0
3,3,5600,5600,5600.0,60 months,21.28,152.39,F,F2,NaN,...,Cash,N,NaN,2011,0,5,11,3,5,0
4,4,5375,5375,5350.0,60 months,12.69,121.45,B,B5,Starbucks,...,Cash,N,NaN,2011,0,5,9,4,1,5


## **Selección de Variables predictoras**   

**Para la seleccion de caracteristicas que se utilizarán en el modelo, la mejor opción es comprender los datos, y comprender las reglas del negocio**

Selecionamos las siguientes variables predictoras (columnas)

- **funded_amnt**             ( monto financiado)  La cantidad total comprometida con ese préstamo en ese momento.
- **loan_term_year**          ( Plazo del credito )
- **int_rate**                ( taza de interes )
- **grade_code**              ( codigo de la calificacion del crédito )
- **purpose_code**            ( codigo del proposito)
- **addr_state_code**         ( codigo de la ciudad)
- **home_ownership_code**     ( codigo situacion casa)
- **annual_inc**              ( ingresos anuales)
- **dti**                     ( Una relación calculada utilizando los pagos de deuda mensuales totales del prestatario )
- **revol_util**              ( cantidad de crédito que el prestatario está utilizando en relación con todo el crédito  )
- **pub_rec_bankruptcies**    ( Número de quiebras de registros públicos )

**Variable dependiente (variable a predecir)**
* **la etiqueta `repaid` será la elegida para el modelo de clasificación**.

**Otros aspectos a considerar**

- Solo las **características que están disponibles antes de que se inicie el préstamo pueden usarse en la clasificación**.
- Las características como **recoveries** (recuperaciones o cargo posterior a la recuperación bruta), **total_rec_prncp** (Principal recibido a la fecha), que solo están disponibles después de que se cierra el préstamo, no deben incluirse en las funciones de entrenamiento.
- Si la clasificación logra una tasa de precisión cercana al 100%, es probable que incluya características que solo estarán disponibles después de que se cierre el préstamo.



In [61]:
X = prestamos_df[['funded_amnt', "int_rate", "grade_code", 'purpose_code', 'addr_state_code',
                 'home_ownership_code', 'annual_inc', 'dti', 'revol_util',
                 'pub_rec_bankruptcies']]

# Variable objetivo o variable a predecir
y = prestamos_df["repaid"]

### **Dividir dataset en conjunto de entrenamiento y prueba**

In [62]:
# Dividimos el dataFrame df en df_train y df_test.
# 60% para dataset de entrenamient0, y 40% para dataset de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [63]:
# verificamos la cantidad de registros asignados al dataframe de entrenamiento
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11944, 10), (7964, 10), (11944,), (7964,))

## **Regresión logística sin balanceo de clases**   
- Construir un modelo de regresión logística con el atributo **class_weight** predeterminado.
- Entrenar y evaluar el modelo.


In [64]:
# Creamoos el clasificador de regresion logistica
clf1 = LogisticRegression(random_state=0)

# Entrenamos el clasificador
clf1.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression(random_state=0)

In [65]:
# verificamos la precisión del modelo en la fase de entrenamiento

print("Precision del clasificador en fase de entrenamiento",
      clf1.score(X_train, y_train) )

Precision del clasificador en fase de entrenamiento 0.8548225050234427


## **Realizar la fase de prueba con los datos de prueba y evaluación del modelo**

In [66]:
# Realizar una prediccion con los datos de prueba
y_pred = clf1.predict(X_test)

# Crear un informe de texto que muestre las principales métricas de clasificación.
print("\nReporte de métricas del clasificador: \n",
      classification_report(y_test, y_pred, target_names=["No Pagado", "Pagado"]))

# Coeficiente de determinación
print( "Precisión:", clf1.score(X_test, y_test) )


Reporte de métricas del clasificador: 
               precision    recall  f1-score   support

   No Pagado       0.08      0.00      0.00      1220
      Pagado       0.85      1.00      0.92      6744

    accuracy                           0.85      7964
   macro avg       0.46      0.50      0.46      7964
weighted avg       0.73      0.85      0.78      7964

Precisión: 0.845429432446007


## **Interpretacion del reporte de Clasificación**  

| Clase     | Precisión (precision) | Sensibilidad (recall) | F1-score | Soporte (support) |
| --------- | --------------------- | --------------------- | -------- | ----------------- |
| No Pagado | 0.08                  | 0.00                  | 0.00     | 1220              |
| Pagado    | 0.85                  | 1.00                  | 0.92     | 6744              |  

A primera vista, la precision parece alta (85%), pero no refleja el verdadero desempeño del modelo, porque las clases están desequilibradas:  
* 6,744 préstamos Pagados  
* 1,220 préstamos No Pagados  

* **El recall de “Pagado” = 1.00** (predice casi todos los pagados correctamente)
* **El recall de “No Pagado” = 0.00** (no logra detectar préstamos impagos)
* **El accuracy alto (0.85)** se debe a que el 85% de los datos pertenecen a la clase mayoritaria (“Pagado”).  

**En otras palabras:**
* El modelo aprendió que si siempre dice “Pagado”, acierta el 85% de las veces, pero **no sirve para detectar los que no pagan (que son justo los más importantes en este contexto)**.



##  **Matriz de confusión**

- Una vez que un modelo es entrenado, la **evaluación** del mismo, **proporciona retroalimentación crítica sobre las características de rendimiento del modelo entrenado**.
- Ayuda a comprender **qué instancias de datos** se están clasificando o **prediciendo incorrectamente**.
- Permite **valorar que tan "bueno" es un modelo de clasificación** basado en aprendizaje automático.
- Muestra los tipos de predicciones correctas e incorrectas que realiza el clasificador.
- La **diagonal principal contiene la suma de todas las predicciones correctas**.
- La **diagonal secundaria refleja los errores del clasificador**: los falsos positivos y los falsos negativos.

#### Matriz de confusión
| Negativo (N) | Positivo (P) |
| --------|---------|
|    TN |    FP   |
|   FN |  TP |


In [67]:
matriz_sin_balanceo = confusion_matrix(y_test, y_pred )
print(f'Matriz Confusion:\n', matriz_sin_balanceo)

Matriz Confusion:
 [[   1 1219]
 [  12 6732]]


## **Observaciones de resultados de la matriz de confusión**  


|                     | Predijo: No Pagado          | Predijo: Pagado                 |
| ------------------- | --------------------------- | ------------------------------- |
| **Real: No Pagado** | **1**  (Verdadero Negativo) | **1219** (Falsos Positivos)     |
| **Real: Pagado**    | **12** (Falsos Negativos)   | **6732** (Verdaderos Positivos) |  

**Clase Pagado**
* **Predicciones correctas (6732)**: casi todos los créditos realmente pagados fueron clasificados correctamente.

**Clase No Pagado**
* **Predicciones correctas (1):** de 1220 créditos realmente impagos, solo 1 fue identificado correctamente :(.
* El modelo confunde casi todos los impagos como “Pagado”.

**Significado en el contexto financiero**  

| Decisión del modelo               | Realidad       | Consecuencia                      |
| --------------------------------- | -------------- | --------------------------------- |
| Predice “Pagado” y sí paga        | Correcto     | Se gana dinero                    |
| Predice “Pagado” pero **no paga** | Error grave  | **Pérdida económica importante**  |
| Predice “No Pagado” y no paga     | Correcto     | Buen filtro de riesgo             |
| Predice “No Pagado” pero sí paga  | Error menor | Se pierde oportunidad de préstamo |  

**Posibles remedios**
* Ajustar pesos de clase
* Rebalancear el dataset: Con SMOTE o undersampling puedes equilibrar las clases artificialmente:




## **Regresión logística con peso de clases balanceado**     

- Debido a que tenemos un **conjunto de datos desbalanceado**, originado porque el 85% de los datos tienen el estatus de pagado, mientras que, el restante 15% tiene el estatus de cancelado.  
- Podemos especificar al **clasificador que use un peso de clases equilibrado** con el método **class_weight='balanced'**.

- Construir un modelo de regresión logística con **class_weight='balanced'**.
- Entrenar y evaluar el modelo.

- Utilizar las mismas caracteristicas que el primer modelo LogisticRegression.  
  
Cómo cambia la puntuación de precisión?


In [68]:
# Crear segundo clasificador con balanceo de clases
clf2 = LogisticRegression(random_state=0, class_weight="balanced")

# Entrenar el nuevo clasificador
clf2.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression(class_weight='balanced', random_state=0)

In [69]:
# verificamos la precisión del modelo 2 en la fase de entrenamiento

print("Exactitud del modelo de entrenamiento:", clf2.score(X_train, y_train))

Exactitud del modelo de entrenamiento: 0.5998827863362358


In [70]:
# Realizar una prediccion con los datos de prueba
y_pred2 = clf2.predict(X_test)

# Crear un informe de texto que muestre las principales métricas de clasificación.
print("\nReporte de métricas del clasificador: \n",
      classification_report(y_test, y_pred2, target_names=["No Pagado", "Pagado"]))

# Coeficiente de determinación
print( "Precisión:", clf2.score(X_test, y_test) )


Reporte de métricas del clasificador: 
               precision    recall  f1-score   support

   No Pagado       0.22      0.63      0.32      1220
      Pagado       0.90      0.58      0.71      6744

    accuracy                           0.59      7964
   macro avg       0.56      0.61      0.51      7964
weighted avg       0.79      0.59      0.65      7964

Precisión: 0.5911602209944752


## **Interpretación**

| Clase         | Precisión | Recall | F1-score | Soporte |
| ------------- | --------- | ------ | -------- | ------- |
| **No Pagado** | 0.22      | 0.63   | 0.32     | 1220    |
| **Pagado**    | 0.90      | 0.58   | 0.71     | 6744    |  

Accuracy global = 0.59 (baja respecto a 0.85 anterior)
pero… ¡eso es una buena señal en este caso

**Después del balanceo:**
* El modelo ya empieza a detectar los impagos (“No Pagado”).
* El recall de “No Pagado” sube de 0.00 → 0.63, esto significa que ahora identifica **63% de los créditos que realmente no se pagan**.
* A cambio, pierde precisión en “Pagado” (de 1.00 → 0.58), lo que es natural.

| Métrica                   | Antes (sin balance) | Ahora (balanceado) | Interpretación                                                   |
| ------------------------- | ------------------- | ------------------ | ---------------------------------------------------------------- |
| **Accuracy**              | 0.85                | 0.59               | Baja, pero ya no engañosa                                        |
| **Recall “No Pagado”**    | 0.00                | 0.63               | Mejora enorme: el modelo detecta impagos                      |
| **Precision “No Pagado”** | 0.08                | 0.22               | También mejora: menos falsas alarmas                             |
| **Recall “Pagado”**       | 1.00                | 0.58               | Baja, el modelo ahora también predice “No Pagado” cuando no debe |
| **F1 “No Pagado”**        | 0.00                | 0.32               | Aumenta (mejor equilibrio entre precisión y recall)              |  

**Conclusiones**  
* Ya distingue entre pagadores y morosos.
* Comete más errores en “Pagado”, pero mejora su utilidad práctica:
  * Antes: rechazaba casi ningún préstamo riesgoso → pérdidas.
  * Ahora: reconoce 6 de cada 10 impagos potenciales → ahorro real.
* En un escenario bancario, **detectar más casos de alto riesgo aunque rechaces algunos buenos clientes es preferible a aprobar todos y perder dinero**.





In [71]:
matriz_balanceo = confusion_matrix(y_test, y_pred2 )
print(f'Matriz Confusion:\n', matriz_balanceo)

Matriz Confusion:
 [[ 769  451]
 [2805 3939]]


### **Interpretación**

|                     | Predijo: No Pagado               | Predijo: Pagado                   |
| ------------------- | -------------------------------- | --------------------------------- |
| **Real: No Pagado** | **769** (Verdaderos Negativos) | **451** (Falsos Positivos)      |
| **Real: Pagado**    | **2805** (Falsos Negativos)    | **3939** (Verdaderos Positivos) |  

**Análisis**
* 769 créditos que realmente no se pagaron fueron correctamente clasificados como No Pagado.  
* 451 créditos sí se pagaron, pero el modelo los clasificó erróneamente como No Pagado.  
* 2805 créditos sí se pagaron, pero el modelo pensó que eran No Pagado.
* 3939 créditos pagados correctamente clasificados como Pagado.  

* **Recall(No Pagado)** =  769 /  769 +  451 ≈ 0.63
* **Recall (Pagado)**   = 3939 / 3939 + 2805 ≈ 0.58



### **Visualizar Importancia de las variables**

In [72]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf2, X_test, y_test, n_repeats=30, random_state=42)

importancia = pd.DataFrame({
    "Variable": X.columns,
    "Importancia Media": result.importances_mean,
    "Desviación": result.importances_std
}).sort_values("Importancia Media", ascending=False)

print(importancia)

               Variable  Importancia Media  Desviación
1              int_rate           0.043508    0.004754
6            annual_inc           0.020714    0.002760
2            grade_code           0.011962    0.002298
5   home_ownership_code           0.009007    0.002229
4       addr_state_code           0.005102    0.002631
7                   dti           0.003721    0.001985
9  pub_rec_bankruptcies          -0.000172    0.000105
8            revol_util          -0.000816    0.000631
3          purpose_code          -0.001126    0.000675
0           funded_amnt          -0.003516    0.001813


In [73]:
# Visualizar importancias con Plotly
import plotly.express as px
fig4 = px.bar(importancia, x="Variable", y="Importancia Media",
              error_y="Desviación", title="Importancia de características (Regresión Logística - Permutation Importance)",
              text_auto=".3f", color="Variable")
fig4.update_layout(width=800, height=600)
fig4.show()

## **Normalizacion de datos**

Muchos algoritmos de machine learning (como regresión logística, SVM, KNN, redes neuronales) son sensibles a la escala de los datos. Si una variable tiene un rango mucho mayor que otra, puede dominar el modelo.  
Por ejemplo:
* Edad: 18 - 70
* Ingresos: 10,000 - 100,000

**Sin escalar, el modelo podría darle más importancia a los ingresos solo por su magnitud.**

In [74]:
from sklearn.preprocessing import StandardScaler

X = prestamos_df[['funded_amnt', "int_rate", "grade_code", 'purpose_code', 'addr_state_code',
                 'home_ownership_code', 'annual_inc', 'dti', 'revol_util',
                 'pub_rec_bankruptcies']]

y = prestamos_df["repaid"]

# stratify es para que mantenga la misma proporción de clases en ambos conjuntos
# Usamos los mismos parametros de tamaño y random usados anteriormente para que sea justa la comparación
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.4, random_state=42, stratify=y)

# Normalizar variables (Regresión Logística (RL) es sensible a la escala)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar modelo RL escalado
RL_scaled = LogisticRegression(random_state=0, class_weight='balanced')
RL_scaled.fit(X_train_scaled, y_train)

# Probar y evaluar el modelo RL escalado
y_pred_scaled = RL_scaled.predict(X_test_scaled)

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_scaled, target_names=["No Pagado", "Pagado"]))


Reporte de clasificación:
              precision    recall  f1-score   support

   No Pagado       0.22      0.61      0.32      1177
      Pagado       0.90      0.62      0.73      6787

    accuracy                           0.61      7964
   macro avg       0.56      0.61      0.53      7964
weighted avg       0.80      0.61      0.67      7964



In [75]:
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_scaled))


Matriz de Confusión:
[[ 719  458]
 [2611 4176]]


## **Comparación de resultados sin normalizar VS con normalización**

| Configuración                    | Verdaderos “No Pagado” | Falsos “No Pagado” | Falsos “Pagado” | Verdaderos “Pagado” | Accuracy |
| -------------------------------- | ---------------------- | ------------------ | --------------- | ------------------- | -------- |
| **Balanceado sin normalizar**    | 769                    | 451                | 2805            | 3939                | ≈ 0.59   |
| **Balanceado con normalización** | 719                    | 458                | 2611            | 4176                | ≈ 0.59   |  
  
  
**¿Por qué no mejora con normalización?**

* Porque la Regresión Logística ya es un modelo lineal y, aunque se beneficia de la normalización (para estabilidad numérica y convergencia), su frontera de decisión no cambia drásticamente cuando las variables están bien escaladas originalmente o tienen rangos comparables.
* Debido a que la Regresión Logística entrega probabilidades, se puede mover el umbral
* Ajustar el umbral de decisión


## **Importancia de las variables despues de escalado**

In [76]:
from sklearn.inspection import permutation_importance

result = permutation_importance(RL_scaled, X_test_scaled, y_test, n_repeats=30, random_state=42)

importancia = pd.DataFrame({
    "Variable": X.columns,
    "Importancia Media": result.importances_mean,
    "Desviación": result.importances_std
}).sort_values("Importancia Media", ascending=False)

print(importancia)

               Variable  Importancia Media  Desviación
1              int_rate           0.039168    0.004611
6            annual_inc           0.009070    0.002748
5   home_ownership_code           0.002712    0.001281
3          purpose_code           0.001666    0.002307
2            grade_code           0.000255    0.000289
4       addr_state_code          -0.000184    0.002106
7                   dti          -0.000209    0.000735
0           funded_amnt          -0.001272    0.002074
9  pub_rec_bankruptcies          -0.002017    0.000916
8            revol_util          -0.002428    0.001027


In [77]:
# Visualizar importancias con Plotly
import plotly.express as px
fig4 = px.bar(importancia, x="Variable", y="Importancia Media",
              error_y="Desviación", title="Importancia de características (Regresión Logística - Permutation Importance)",
              text_auto=".3f", color="Variable")
fig4.update_layout(width=800, height=600)
fig4.show()

## **Ajustando el umbral de decisión**  

* El umbral de decisión (o threshold) es el valor a partir del cual decides si una probabilidad se convierte en una clase positiva (1) o negativa (0).
* Por defecto, este umbral es 0.5:
  * Si la probabilidad es ≥ 0.5, se predice clase 1.
  * Si es < 0.5, se predice clase 0.

Cambiar el umbral permite ajustar la sensibilidad y especificidad del modelo según el problema. Por ejemplo:

* **Si detectar "No Pagado"** es más importante (porque implica riesgo financiero), se puede bajar el umbral (por ejemplo, a 0.35). Así, el modelo será más propenso a predecir "No Pagado", aunque aumente el número de falsos positivos.
* **Si prefieres menos falsos positivos**, puedes subir el umbral, pero corres el riesgo de pasar por alto casos importantes.
* Para elegir el mejor umbral se puede usar métricas como:
  * **Matriz de confusión**: para analizar falsos positivos y negativos.
  * **F1-score**: si quieres un balance entre precisión y recall.
  * **Curva ROC y AUC**: para ver el rendimiento del modelo a diferentes umbrales

In [78]:
# Ajustar el umbral de decisión a 0.45

# Predicción con umbral ajustado
# Extraer probabilidades de clase positiva (Pagado = 1)
y_prob = RL_scaled.predict_proba(X_test_scaled)[:, 1]

# Ajustar el umbral de decisión a 0.45
umbral = 0.45   # ↓ bajarlo aumenta detección de "No Pagado"
y_pred_umbral = (y_prob >= umbral).astype(int)

# Evaluación del modelo
print(f"\nReporte de clasificación con umbral = {umbral}")
print(classification_report(y_test, y_pred_umbral, target_names=["No Pagado", "Pagado"]))

print("\nMatriz de Confusión con umbral de decisión a 0.45:")
print(confusion_matrix(y_test, y_pred_umbral))


Reporte de clasificación con umbral = 0.45
              precision    recall  f1-score   support

   No Pagado       0.24      0.50      0.32      1177
      Pagado       0.89      0.72      0.80      6787

    accuracy                           0.69      7964
   macro avg       0.57      0.61      0.56      7964
weighted avg       0.80      0.69      0.73      7964


Matriz de Confusión con umbral de decisión a 0.45:
[[ 587  590]
 [1880 4907]]


In [79]:
# Ajustar el umbral de decisión a 0.40

# Predicción con umbral ajustado
# Extraer probabilidades de clase positiva (Pagado = 1)
y_prob = RL_scaled.predict_proba(X_test_scaled)[:, 1]

# Ajustar el umbral de decisión a 0.40
umbral = 0.40   # ↓ bajarlo aumenta detección de "No Pagado"
y_pred_umbral = (y_prob >= umbral).astype(int)

# Evaluación del modelo
print(f"\nReporte de clasificación con umbral = {umbral}")
print(classification_report(y_test, y_pred_umbral, target_names=["No Pagado", "Pagado"]))

print("\nMatriz de Confusión con umbral de decisión a 0.40:")
print(confusion_matrix(y_test, y_pred_umbral))


Reporte de clasificación con umbral = 0.4
              precision    recall  f1-score   support

   No Pagado       0.26      0.37      0.30      1177
      Pagado       0.88      0.82      0.85      6787

    accuracy                           0.75      7964
   macro avg       0.57      0.59      0.58      7964
weighted avg       0.79      0.75      0.77      7964


Matriz de Confusión con umbral de decisión a 0.40:
[[ 433  744]
 [1251 5536]]


In [80]:
# Ajustar el umbral de decisión a 0.55

# Predicción con umbral ajustado
# Extraer probabilidades de clase positiva (Pagado = 1)
y_prob = RL_scaled.predict_proba(X_test_scaled)[:, 1]

# Ajustar el umbral de decisión a 0.55
umbral = 0.55   # ↓ bajarlo aumenta detección de "No Pagado"
y_pred_umbral = (y_prob >= umbral).astype(int)

# Evaluación del modelo
print(f"\nReporte de clasificación con umbral = {umbral}")
print(classification_report(y_test, y_pred_umbral, target_names=["No Pagado", "Pagado"]))

print("\nMatriz de Confusión con umbral de decisión a 0.55")
print(confusion_matrix(y_test, y_pred_umbral))


Reporte de clasificación con umbral = 0.55
              precision    recall  f1-score   support

   No Pagado       0.21      0.76      0.33      1177
      Pagado       0.92      0.50      0.65      6787

    accuracy                           0.54      7964
   macro avg       0.57      0.63      0.49      7964
weighted avg       0.82      0.54      0.60      7964


Matriz de Confusión con umbral de decisión a 0.55
[[ 890  287]
 [3389 3398]]


In [81]:
# Ajustar el umbral de decisión a 0.60

# Predicción con umbral ajustado
# Extraer probabilidades de clase positiva (Pagado = 1)
y_prob = RL_scaled.predict_proba(X_test_scaled)[:, 1]

# Ajustar el umbral de decisión a 0.60
umbral = 0.60   # ↓ bajarlo aumenta detección de "No Pagado"
y_pred_umbral = (y_prob >= umbral).astype(int)

# Evaluación del modelo
print(f"\nReporte de clasificación con umbral = {umbral}")
print(classification_report(y_test, y_pred_umbral, target_names=["No Pagado", "Pagado"]))

print("\nMatriz de Confusión con umbral de decisión a 0.60")
print(confusion_matrix(y_test, y_pred_umbral))


Reporte de clasificación con umbral = 0.6
              precision    recall  f1-score   support

   No Pagado       0.19      0.84      0.31      1177
      Pagado       0.93      0.38      0.54      6787

    accuracy                           0.45      7964
   macro avg       0.56      0.61      0.43      7964
weighted avg       0.82      0.45      0.51      7964


Matriz de Confusión con umbral de decisión a 0.60
[[ 988  189]
 [4204 2583]]


## **Interpretacion de resultados**

|  Umbral  | Verdaderos No Pagados (TP “No Pagado”) | Verdaderos Pagados |                      Recall “No Pagado”                     |       Recall “Pagado”       | Comentario                                                                            |
| :------: | :------------------------------------: | :----------------: | :---------------------------------------------------------: | :-------------------------: | :------------------------------------------------------------------------------------ |
| **0.60** |                   988                  |        2583        | Mejor en “No Pagado” pero muy pocos aciertos en “Pagado” |  Mucho peor en “Pagado”  | El modelo se vuelve **más estricto** (predice “Pagado” solo con probabilidad alta).   |
| **0.55** |                   890                  |        3398        |             Aumenta detección de “No Pagado”             | Pierde algo de precisión | Aún estricto, pero balancea un poco.                                                  |
| **0.45** |                   587                  |        4907        |                  Buen equilibrio general                 |   Mejor precisión total  | Cerca del punto de equilibrio clásico (0.5).                                          |
| **0.40** |                   433                  |        5536        |                 Detecta menos “No Pagado”                |  Más “Pagados” correctos | El modelo se vuelve **más permisivo** → detecta más pagos, pero deja escapar impagos. |  

### **Conclusiones**
* Cuando se baja el umbral (ej. 0.4), el modelo predice más “Pagado” → mejora la tasa de aciertos de pagos, pero pierde sensibilidad a los impagos.
* Cuando se sube el umbral (ej. 0.6), el modelo se vuelve más exigente para decir “Pagado” → detecta más impagos, pero también marca como “No Pagado” muchos préstamos que sí se pagaron.

En los casos analizados, el umbral entre 0.45 y 0.50 logra un buen equilibrio entre:
* detectar un número razonable de impagos, sin comprometer demasiado la precisión global (accuracy cercano al 0.6).


### **Implementación del modelo para Predicciones sobre nuevos valores  (ficticios)**

In [82]:
# Debemos proporcionar 10 valores (variables predictoras)

yhat_predmanual =  clf2.predict( [[5000, 15.5 , 5, 3, 4, 4, 30000, 12, 12, 0 ]] )

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



In [83]:
# Resultado de la  prediccion manual
# El resultado es 0,  es decir, que el prestamo no se paga
yhat_predmanual

array([0])